In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [3]:

import chardet
path = '../LR2/spotify-2023.csv'
with open(path, 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

data = pd.read_csv(path, encoding=encoding)
data = data.drop(574)
data = data.dropna()
print(data.count());

track_name              816
artist(s)_name          816
artist_count            816
released_year           816
released_month          816
released_day            816
in_spotify_playlists    816
in_spotify_charts       816
streams                 816
in_apple_playlists      816
in_apple_charts         816
in_deezer_playlists     816
in_deezer_charts        816
in_shazam_charts        816
bpm                     816
key                     816
mode                    816
danceability_%          816
valence_%               816
energy_%                816
acousticness_%          816
instrumentalness_%      816
liveness_%              816
speechiness_%           816
dtype: int64


# Было решено предсказывать danceability

## Задание 2

### ввиду того, что предсказываем процент danceability, то очевдно, что это задача регрессии

In [4]:
selected_features = ['artist_count', 'released_year', 'streams', 'in_apple_playlists']
target_variable = 'danceability_%'
X = data[selected_features]
X['streams']=X['streams'].astype(float)
X['artist_count']=X['artist_count'].astype(float)
X['released_year']=X['released_year'].astype(float)
X['in_apple_playlists']=X['in_apple_playlists'].astype(float)
Y = data[target_variable]

/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_23188/1403072990.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['streams']=X['streams'].astype(float)
/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_23188/1403072990.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['artist_count']=X['artist_count'].astype(float)
/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_23188/1403072990.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=17)

In [6]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train[selected_features])

X_test_scaled = scaler.transform(X_test[selected_features])

X.loc[X_train.index, selected_features] = X_train_scaled
X.loc[X_test.index, selected_features] = X_test_scaled




In [7]:
X.std()

artist_count          1.028629
released_year         1.013628
streams               1.012255
in_apple_playlists    1.037737
dtype: float64

In [8]:
X['streams'].mean()

-0.02232481104313965

# Задание 4, 5

In [9]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.linear_model import SGDRegressor

from sklearn.metrics import mean_absolute_error,  mean_squared_error



In [10]:
def getMetrics(test,predict):
    MAE  = mean_absolute_error(test, predict)
    print(f"MAE: {MAE:.2f}")

    RMSE = np.sqrt(mean_squared_error(test, predict))
    print(f"RMSE: {RMSE:.2f}")

    MAPE = np.mean(np.abs((test - predict) / test)) * 100
    print(f"MAPE: {MAPE:.2f}%")

## тест сет

обучить на трейне

In [13]:
linClassifier  = LinearRegression();

print(f"{'Тест сет':–^25}")


linClassifier.fit(X_train_scaled, y_train)
lr_pred = linClassifier.predict(X_test_scaled)
getMetrics(y_test, lr_pred)

print(f"{'Трейн сет':–^25}")
linClassifier  = LinearRegression();
linClassifier.fit(X_train_scaled, y_train)

logr_pred_train = linClassifier.predict(X_train_scaled)

getMetrics(y_train, logr_pred_train)


––––––––Тест сет–––––––––
MAE: 12.75
RMSE: 15.76
MAPE: 22.00%
––––––––Трейн сет––––––––
MAE: 10.96
RMSE: 13.54
MAPE: 18.68%


## Попытки улучшить модель

In [15]:
from sklearn.preprocessing import PolynomialFeatures


for power in range(1,8):
    poly = PolynomialFeatures(degree=power);

    X_poly = poly.fit_transform(X[selected_features])
    regr = LinearRegression()

    regr.fit(X_poly, Y)
    pred = regr.predict(poly.transform(X_test_scaled))
    print(f"{f'Степень {power}':–^25}")
    getMetrics(y_test, pred)

plt.show()

/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


––––––––Степень 1––––––––
MAE: 12.71
RMSE: 15.63
MAPE: 22.02%
––––––––Степень 2––––––––
MAE: 12.43
RMSE: 15.45
MAPE: 21.56%
––––––––Степень 3––––––––
MAE: 11.65
RMSE: 14.64
MAPE: 20.20%
––––––––Степень 4––––––––
MAE: 10.99
RMSE: 13.97
MAPE: 19.06%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


––––––––Степень 5––––––––
MAE: 9.96
RMSE: 13.16
MAPE: 17.38%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


––––––––Степень 6––––––––
MAE: 8.97
RMSE: 12.42
MAPE: 15.65%
––––––––Степень 7––––––––
MAE: 8.30
RMSE: 11.86
MAPE: 14.69%


/Users/zanuragin03/Desktop/Учеба/ML/venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
